## 0. Libarary 불러오기 및 경로설정

In [1]:
import os
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize

import timm
from timm import create_model

from tqdm import tqdm

import numpy as np

In [2]:
# 테스트 데이터셋 폴더 경로를 지정해주세요.
test_dir = '/opt/ml/input/data/eval'

## 1. Model 정의

In [3]:
class KFoldClassifier(nn.Module):
    def __init__(self, num_classes, load_model):
        super(KFoldClassifier, self).__init__()

        self.num_classes = num_classes
        self.load_model = load_model
        if self.load_model:
            self.backbone = create_model(self.load_model, pretrained=True, num_classes=self.num_classes)
            

    def forward(self, x):
        if self.load_model:
            x = self.backbone(x)
        return x

## 2. Test Dataset 정의

In [4]:
class TestDataset(Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        return image

    def __len__(self):
        return len(self.img_paths)

## 3. Inference

In [5]:
# meta 데이터와 이미지 경로를 불러옵니다.
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
image_dir = os.path.join(test_dir, 'images')

# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]
transform = transforms.Compose([transforms.CenterCrop((300, 300)),
                                transforms.Resize((224, 224)),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=(0.485, 0.456, 0.406),
                                                     std=(0.229, 0.224, 0.225))])
dataset = TestDataset(image_paths, transform)

batch_size = 64
loader = DataLoader(
    dataset,
    shuffle=False,
    batch_size=batch_size
)

# 모델을 정의합니다. (학습한 모델이 있다면 torch.load로 모델을 불러주세요!)
mask_ckp = ['/opt/ml/level1_imageclassification-cv-04/checkpoint/kfold4_0_cd_maskdetection_reducelr61_bs64_ep100_adamw_lr0.0001_resnet50/epoch(39)_acc(0.984)_loss(0.057)_f1(0.984)_state_dict.pt',
            '/opt/ml/level1_imageclassification-cv-04/checkpoint/kfold4_1_cd_maskdetection_reducelr67_bs64_ep100_adamw_lr0.0001_resnet50/epoch(39)_acc(0.991)_loss(0.039)_f1(0.991)_state_dict.pt',
            '/opt/ml/level1_imageclassification-cv-04/checkpoint/kfold4_2_cd_maskdetection_reducelr69_bs64_ep100_adamw_lr0.0001_resnet50/epoch(54)_acc(0.991)_loss(0.051)_f1(0.991)_state_dict.pt',
            '/opt/ml/level1_imageclassification-cv-04/checkpoint/kfold4_3_cd_maskdetection_reducelr73_bs64_ep100_adamw_lr0.0001_resnet50/epoch(49)_acc(0.991)_loss(0.032)_f1(0.991)_state_dict.pt',
            '/opt/ml/level1_imageclassification-cv-04/checkpoint/kfold4_4_cd_maskdetection_reducelr76_bs64_ep100_adamw_lr0.0001_resnet50/epoch(29)_acc(0.992)_loss(0.043)_f1(0.992)_state_dict.pt']
gender_ckp = ['/opt/ml/level1_imageclassification-cv-04/checkpoint/kfold4_0_cd_genderdetection_reducelr66_bs64_ep100_adamw_lr0.0001_resnet50/epoch(56)_acc(0.944)_loss(0.158)_f1(0.941)_state_dict.pt',
              '/opt/ml/level1_imageclassification-cv-04/checkpoint/kfold4_1_cd_genderdetection_reducelr70_bs64_ep100_adamw_lr0.0001_resnet50/epoch(19)_acc(0.965)_loss(0.114)_f1(0.963)_state_dict.pt',
              '/opt/ml/level1_imageclassification-cv-04/checkpoint/kfold4_2_cd_genderdetection_reducelr70_bs64_ep100_adamw_lr0.0001_resnet50/epoch(19)_acc(0.967)_loss(0.111)_f1(0.966)_state_dict.pt',
              '/opt/ml/level1_imageclassification-cv-04/checkpoint/kfold4_3_cd_genderdetection_reducelr74_bs64_ep100_adamw_lr0.0001_resnet50/epoch(19)_acc(0.953)_loss(0.137)_f1(0.950)_state_dict.pt',
              '/opt/ml/level1_imageclassification-cv-04/checkpoint/kfold4_4_cd_genderdetection_reducelr77_bs64_ep100_adamw_lr0.0001_resnet50/epoch(42)_acc(0.923)_loss(0.216)_f1(0.919)_state_dict.pt']
age_ckp = ['/opt/ml/level1_imageclassification-cv-04/checkpoint/kfold4_0_age_swin80_bs64_ep100_adamw_lr1e-06_swin_base_patch4_window7_224_in22k/epoch(39)_acc(0.878)_loss(0.323)_f1(0.839)_state_dict.pt',
           '/opt/ml/level1_imageclassification-cv-04/checkpoint/kfold4_1_age_swin81_bs64_ep100_adamw_lr1e-06_swin_base_patch4_window7_224_in22k/epoch(29)_acc(0.866)_loss(0.303)_f1(0.820)_state_dict.pt']
        #    '/opt/ml/level1_imageclassification-cv-04/checkpoint/kfold4_2_age_swin85_bs64_ep100_adamw_lr1e-07_swin_base_patch4_window7_224_in22k/epoch(79)_acc(0.842)_loss(0.361)_f1(0.794)_state_dict.pt',
        #    '/opt/ml/level1_imageclassification-cv-04/checkpoint/kfold4_3_age_swin86_bs64_ep100_adamw_lr1e-07_swin_base_patch4_window7_224_in22k/epoch(99)_acc(0.814)_loss(0.388)_f1(0.770)_state_dict.pt',
        #    '/opt/ml/level1_imageclassification-cv-04/checkpoint/kfold4_4_age_swin98_bs64_ep200_adamw_lr1e-06_swin_base_patch4_window7_224_in22k/epoch(9)_acc(0.806)_loss(0.533)_f1(0.739)_state_dict.pt']

# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
all_ckps = [mask_ckp, gender_ckp, age_ckp]
num_classes = [3, 2, 3]
load_model = ['resnet50', 'resnet50', 'swin_base_patch4_window7_224_in22k']
device = torch.device('cuda')
predictions = [[], [], []]

for i, (ckps, num_class) in enumerate(zip(all_ckps, num_classes)):
    models = []
    for ckp in ckps:
        state_dict = torch.load(ckp)
        model = KFoldClassifier(num_class, load_model[i]).to(device)
        model.load_state_dict(state_dict['model_state_dict'])
        model.eval()
        models.append(model)

    for images in tqdm(loader):
        with torch.no_grad():
            images = images.to(device)
            # class_pred = torch.zeros((batch_size, num_class)).to(device)
            class_pred = []
            for model in models:
                pred = model(images)
                class_pred.append(pred)
            all_class_pred = torch.stack(class_pred)
            class_pred = torch.sum(all_class_pred, dim=0)
            pred = class_pred.argmax(dim=-1)
            predictions[i].extend(pred.cpu().numpy())

predictions = np.array(predictions)
all_predictions = predictions[0]*6 + predictions[1]*3 + predictions[2]
submission['ans'] = all_predictions

# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join(test_dir, 'submission.csv'), index=False)
print('test inference is done!')

100%|██████████| 197/197 [09:52<00:00,  3.01s/it]

test inference is done!
